In [77]:
import sys
import os

# os.getcwd() => '/home/genie/Documents/Projekti/cash-flow/jupyter'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Moves up one level to 'cash-flow'
# project_root = '/home/genie/Documents/Projekti/cash-flow'

# Add the module Converters directory to sys.path
sys.path.append(os.path.join(project_root, 'cash_flow', 'util'))


In [78]:
# Imports and engine

import pandas as pd
import numpy as np
import datetime
from sqlalchemy import create_engine

from Converters import date_format

engine = create_engine("sqlite:///../data/database.db", echo=False)

In [79]:
# Setup filters
date_from = "2024-01-01"
date_through = "2025-12-31"
report_period = 'W-SUN'

In [80]:
# Load all data from database

bank_df = pd.read_sql_query('SELECT * FROM D13_CF_Bank_Union WHERE d_date >= "' + date_from + '" AND d_date <= "' + date_through + '" ', engine)
cash_df = pd.read_sql_query('SELECT * FROM D10_Cash_Transactions WHERE d_date <= "' + date_through + '" ', engine)
definition_df = pd.read_sql_table("E01_CashFlowDefinition", engine)
definition_df.rename(columns={"id":"definition_id"}, inplace=True)
definition_acc_df = definition_df[definition_df["definition_type"] == 1]
definition_tot_df = definition_df[definition_df["definition_type"] == 2]
definition_bal_df = definition_df[definition_df["definition_type"] == 3]
definition_accounts_df = pd.read_sql_table("E01_CashFlowDefinitionAccounts", engine)
definition_totals_df = pd.read_sql_table("E01_CashFlowDefinitionTotals", engine)


In [81]:
bank_df.head()

,d_id,d_type,d_date,d_customer,d_vendor,gl_account,gl_entry_type,gl_amount,d_currency,gl_amount_LC
0,1504,1,2024-11-06 00:00:00.000000,76.0,NaN,2310,CR,5011.69,EUR,5011.69
1,1504,1,2024-11-06 00:00:00.000000,76.0,NaN,2620,DR,5011.69,EUR,5011.69
2,1505,1,2025-11-15 00:00:00.000000,69.0,NaN,2310,DR,0.00,EUR,0.00
3,1505,1,2025-11-15 00:00:00.000000,69.0,NaN,2620,DR,9510.12,EUR,9510.12
4,1505,1,2025-11-15 00:00:00.000000,69.0,NaN,5721,CR,1650.52,EUR,1650.52


In [82]:
cash_df.head()

,id,d_id,d_type,d_number,d_date,d_vendor,d_customer,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC
0,6568,1784,1,1284,2020-01-02 00:00:00.000000,NaN,58.0,Payment 1284,EUR,DR,2620,2549.68,2549.68
1,6002,1501,1,1001,2020-01-04 00:00:00.000000,NaN,38.0,Payment 1001,EUR,DR,2620,9894.95,9894.95
2,15233,4317,2,3017,2020-01-05 00:00:00.000000,24.0,NaN,Payment 3017,EUR,CR,2620,3719.79,3719.79
3,15811,4606,2,3306,2020-01-07 00:00:00.000000,67.0,NaN,Payment 3306,EUR,CR,2620,2403.99,2403.99
4,15891,4646,2,3346,2020-01-07 00:00:00.000000,71.0,NaN,Payment 3346,EUR,CR,2620,2731.05,2731.05


In [83]:
definition_acc_df.head()

,definition_id,key,definition_type,name
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas
2,3,102,1.0,Maksājumi piegādātājiem
3,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi
5,7,201,1.0,Izdevumi procentu maksājumiem
6,8,202,1.0,Izdevumi nodokļu maksājumiem


In [84]:
definition_accounts_df.head()

,id,definition_id,operator,entry_type,account
0,1,31,+,DR,7210


In [85]:
definition_tot_df.head()

,definition_id,key,definition_type,name
4,6,199,2.0,Bruto pamatdarbības naudas plūsma
8,10,299,2.0,Pamatdarbības neto naudas plūsma
18,20,309,2.0,Ieguldījumu darbības neto naudas plūsma
25,27,499,2.0,Finansēšanas darbības neto naudas plūsma
27,29,902,2.0,Neto naudas plūsma


In [86]:
definition_totals_df.head()

,id,definition_id,operator,definition_summarized


In [87]:
definition_bal_df.head()

,definition_id,key,definition_type,name
28,30,999,3.0,Naudas līdzekļu atlikums pārskata perioda beigās


In [88]:
# ********************** (1) Start working with sums based on filtered accounts *******************************
# Merge accounts into accounts definition

definition_accounts_df = pd.merge(
    definition_acc_df,
    definition_accounts_df,
    on="definition_id", how="left")
definition_accounts_df.drop(columns=["id"], inplace=True)

In [89]:
definition_accounts_df

,definition_id,key,definition_type,name,operator,entry_type,account
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,NaN,NaN,NaN
1,3,102,1.0,Maksājumi piegādātājiem,NaN,NaN,NaN
2,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,NaN,NaN,NaN
3,7,201,1.0,Izdevumi procentu maksājumiem,NaN,NaN,NaN
4,8,202,1.0,Izdevumi nodokļu maksājumiem,NaN,NaN,NaN
5,9,203,1.0,Naudas plūsma no ārkārtas posteņiem,NaN,NaN,NaN
6,12,301,1.0,Radniecīgo vai asociēto uzņēmumu daļu iegāde,NaN,NaN,NaN
7,13,302,1.0,Ieņēmumi no radniecīgo vai asociēto uzņēmumu d...,NaN,NaN,NaN
8,14,303,1.0,Pamatlīdzekļu un nemateriālo ieguldījumu iegāde,NaN,NaN,NaN
9,15,304,1.0,Ieņēmumi no pamatlīdzekļu un nemateriālo iegul...,NaN,NaN,NaN


In [90]:
# Merge accounts definition into bank transactions

merged_df = pd.merge(definition_accounts_df,
    bank_df,
    left_on=['entry_type', 'account'],        # Columns in definition_df
    right_on=['gl_entry_type', 'gl_account'], # Corresponding columns in transactions_df
    how='left'
)


In [91]:
# Add column adjusted_amount_LC based on operator + / -

merged_df['adjusted_amount_LC'] = np.where(
    merged_df['operator'] == '+', 
    merged_df['gl_amount_LC'], 
    -merged_df['gl_amount_LC']
)


In [92]:
# Convert d_date to_datetime if not done yet

merged_df['d_date'] = pd.to_datetime(merged_df['d_date'])

# Convert adjusted_amount_LC to_numeric if not done yet

merged_df['adjusted_amount_LC'] = pd.to_numeric(merged_df['adjusted_amount_LC'], errors='coerce')

In [93]:
# Add column for period reference

merged_df["d_period"] = merged_df['d_date'].dt.to_period(report_period).apply(lambda r: r.to_timestamp(how='end').normalize() if pd.notna(r) else pd.NaT)

In [94]:
merged_df.head()

,definition_id,key,definition_type,name,operator,entry_type,account,d_id,d_type,d_date,d_customer,d_vendor,gl_account,gl_entry_type,gl_amount,d_currency,gl_amount_LC,adjusted_amount_LC,d_period
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,3,102,1.0,Maksājumi piegādātājiem,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,7,201,1.0,Izdevumi procentu maksājumiem,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,8,202,1.0,Izdevumi nodokļu maksājumiem,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [95]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   definition_id       139 non-null    int64         
 1   key                 139 non-null    object        
 2   definition_type     139 non-null    float64       
 3   name                139 non-null    object        
 4   operator            119 non-null    object        
 5   entry_type          119 non-null    object        
 6   account             119 non-null    object        
 7   d_id                119 non-null    float64       
 8   d_type              119 non-null    float64       
 9   d_date              119 non-null    datetime64[ns]
 10  d_customer          0 non-null      float64       
 11  d_vendor            119 non-null    float64       
 12  gl_account          119 non-null    object        
 13  gl_entry_type       119 non-null    object        

In [96]:
# Pivot values based on definition_id and d_period

pivot_df = merged_df.pivot_table(
    index='definition_id',
    columns='d_period',
    values='adjusted_amount_LC',
    aggfunc="sum",
    fill_value=0,
    dropna=False
)


In [97]:
# Add all periods in range if some are missing

all_periods = pd.date_range(start=date_from, end=date_through, freq=report_period)
pivot_df = pivot_df.reindex(columns=all_periods, fill_value=0)


In [129]:
# Ensure columns are in datetime format before formatting
pivot_df.columns = pd.to_datetime(pivot_df.columns, errors='coerce')



In [99]:
pivot_df

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
2,0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,...,0.00,0,0,0.00,0.00,0,0.00,0.00,0,0.00
3,0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,...,0.00,0,0,0.00,0.00,0,0.00,0.00,0,0.00
5,0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,...,0.00,0,0,0.00,0.00,0,0.00,0.00,0,0.00
7,0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,...,0.00,0,0,0.00,0.00,0,0.00,0.00,0,0.00
8,0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,...,0.00,0,0,0.00,0.00,0,0.00,0.00,0,0.00
9,0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,...,0.00,0,0,0.00,0.00,0,0.00,0.00,0,0.00
12,0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,...,0.00,0,0,0.00,0.00,0,0.00,0.00,0,0.00
13,0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,...,0.00,0,0,0.00,0.00,0,0.00,0.00,0,0.00
14,0,0.00,0.00,0.00,0,0,0,0.00,0.00,0.00,...,0.00,0,0,0.00,0.00,0,0.00,0.00,0,0.00


In [100]:
# ********************** (2) Start working with totals based on totaled definitions *******************************
# Merge totals into totals definition
definition_totals_df = pd.merge(
    definition_tot_df,
    definition_totals_df,
    on="definition_id", how="left")
definition_totals_df.drop(columns=["id"], inplace=True)


In [101]:
definition_totals_df

,definition_id,key,definition_type,name,operator,definition_summarized
0,6,199,2.0,Bruto pamatdarbības naudas plūsma,NaN,NaN
1,10,299,2.0,Pamatdarbības neto naudas plūsma,NaN,NaN
2,20,309,2.0,Ieguldījumu darbības neto naudas plūsma,NaN,NaN
3,27,499,2.0,Finansēšanas darbības neto naudas plūsma,NaN,NaN
4,29,902,2.0,Neto naudas plūsma,NaN,NaN


In [102]:
# Merge totals definition with summarized accounts pivot

merged_totals_df = pd.merge(definition_totals_df,
    pivot_df,
    left_on='definition_summarized',
    right_on='definition_id', 
    how='left'
)

# Add multiplication num for operator

merged_totals_df['op_num'] = merged_totals_df['operator'].map({'+': 1, '-': -1})

# Drop unnecessary columns

merged_totals_df.drop(columns = ['key', 'definition_type', 'name', 'operator', 'definition_summarized'], inplace=True)


In [103]:
merged_totals_df

,definition_id,2024-01-07 00:00:00,2024-01-14 00:00:00,2024-01-21 00:00:00,2024-01-28 00:00:00,2024-02-04 00:00:00,2024-02-11 00:00:00,2024-02-18 00:00:00,2024-02-25 00:00:00,2024-03-03 00:00:00,...,2025-11-02 00:00:00,2025-11-09 00:00:00,2025-11-16 00:00:00,2025-11-23 00:00:00,2025-11-30 00:00:00,2025-12-07 00:00:00,2025-12-14 00:00:00,2025-12-21 00:00:00,2025-12-28 00:00:00,op_num
0,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
# Choose value_columns for further summarization

value_columns = [col for col in merged_totals_df.columns if col not in ['definition_id', 'op_num']]

In [105]:
value_columns

[Timestamp('2024-01-07 00:00:00'),
 Timestamp('2024-01-14 00:00:00'),
 Timestamp('2024-01-21 00:00:00'),
 Timestamp('2024-01-28 00:00:00'),
 Timestamp('2024-02-04 00:00:00'),
 Timestamp('2024-02-11 00:00:00'),
 Timestamp('2024-02-18 00:00:00'),
 Timestamp('2024-02-25 00:00:00'),
 Timestamp('2024-03-03 00:00:00'),
 Timestamp('2024-03-10 00:00:00'),
 Timestamp('2024-03-17 00:00:00'),
 Timestamp('2024-03-24 00:00:00'),
 Timestamp('2024-03-31 00:00:00'),
 Timestamp('2024-04-07 00:00:00'),
 Timestamp('2024-04-14 00:00:00'),
 Timestamp('2024-04-21 00:00:00'),
 Timestamp('2024-04-28 00:00:00'),
 Timestamp('2024-05-05 00:00:00'),
 Timestamp('2024-05-12 00:00:00'),
 Timestamp('2024-05-19 00:00:00'),
 Timestamp('2024-05-26 00:00:00'),
 Timestamp('2024-06-02 00:00:00'),
 Timestamp('2024-06-09 00:00:00'),
 Timestamp('2024-06-16 00:00:00'),
 Timestamp('2024-06-23 00:00:00'),
 Timestamp('2024-06-30 00:00:00'),
 Timestamp('2024-07-07 00:00:00'),
 Timestamp('2024-07-14 00:00:00'),
 Timestamp('2024-07-

In [106]:
# Summarize value_columns based on group value and op_num

summarized_totals_df = (
    merged_totals_df.groupby('definition_id', group_keys=False)[value_columns + ['op_num']]
    .apply(lambda group: pd.Series(
        (group['op_num'].to_numpy()[:, None] * group[value_columns].to_numpy()).sum(axis=0),
        index=value_columns
    ))
    # .reset_index()
)


In [107]:
summarized_totals_df

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [108]:
# ********************** (3) Start working with balances on end of each period *******************************

# Convert d_date to_datetime if not yet
cash_df['d_date'] = pd.to_datetime(cash_df['d_date'])


In [109]:
cash_df.head()

,id,d_id,d_type,d_number,d_date,d_vendor,d_customer,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC
0,6568,1784,1,1284,2020-01-02,NaN,58.0,Payment 1284,EUR,DR,2620,2549.68,2549.68
1,6002,1501,1,1001,2020-01-04,NaN,38.0,Payment 1001,EUR,DR,2620,9894.95,9894.95
2,15233,4317,2,3017,2020-01-05,24.0,NaN,Payment 3017,EUR,CR,2620,3719.79,3719.79
3,15811,4606,2,3306,2020-01-07,67.0,NaN,Payment 3306,EUR,CR,2620,2403.99,2403.99
4,15891,4646,2,3346,2020-01-07,71.0,NaN,Payment 3346,EUR,CR,2620,2731.05,2731.05


In [110]:
cash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             2000 non-null   int64         
 1   d_id           2000 non-null   int64         
 2   d_type         2000 non-null   int64         
 3   d_number       2000 non-null   object        
 4   d_date         2000 non-null   datetime64[ns]
 5   d_vendor       1000 non-null   float64       
 6   d_customer     1000 non-null   float64       
 7   d_description  2000 non-null   object        
 8   d_currency     2000 non-null   object        
 9   gl_entry_type  2000 non-null   object        
 10  gl_account     2000 non-null   object        
 11  gl_amount      2000 non-null   float64       
 12  gl_amount_LC   2000 non-null   float64       
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 203.2+ KB


In [111]:
# Initialize cumulative balance
cumulative_balance = 0  
balances = []

# Start index for efficient filtering
start_idx = 0  

for eoperiod in value_columns:
    # Filter only new transactions from the last weekend up to the current one
    new_transactions = cash_df[(cash_df.index >= start_idx) & (cash_df['d_date'] <= eoperiod)]
    
    # If there are new transactions, update cumulative balance
    if not new_transactions.empty:
        cumulative_balance += new_transactions.apply(
            lambda row: row['gl_amount_LC'] if row['gl_entry_type'] == 'DR' else -row['gl_amount_LC'], axis=1
        ).sum()
        
        # Move the start index forward to avoid redundant calculations
        start_idx = new_transactions.index[-1] + 1  

    # Store balance for the current weekend
    balances.append(cumulative_balance)



In [112]:
# Create a DataFrame with periods as columns
balances_df = pd.DataFrame([balances], columns=value_columns)


In [113]:
balances_df

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
0,1207.71,9797.47,-1107.69,-1457.53,-444.23,-973.86,19349.89,5912.69,20901.93,12846.21,...,-119732.25,-123971.21,-129841.27,-116397.08,-129006.87,-116478.45,-103608.16,-104947.13,-80904.51,-87609.43


In [114]:
# Create a new DataFrame with repeated rows for each definition_id
balances_df = pd.merge(definition_bal_df["definition_id"], balances_df, how='cross')
balances_df = balances_df.set_index("definition_id")


In [115]:
balances_df

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
30,1207.71,9797.47,-1107.69,-1457.53,-444.23,-973.86,19349.89,5912.69,20901.93,12846.21,...,-119732.25,-123971.21,-129841.27,-116397.08,-129006.87,-116478.45,-103608.16,-104947.13,-80904.51,-87609.43


In [116]:
# ********************** (3) Put together summarized accounts, totals and balances on end of each period *******************************

In [117]:
pivot_df.head()

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
2,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0.0
3,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0.0
5,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0.0
7,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0.0
8,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0,...,0.0,0,0,0.0,0.0,0,0.0,0.0,0,0.0


In [118]:
summarized_totals_df.head()

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [119]:
balances_df.head()

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
30,1207.71,9797.47,-1107.69,-1457.53,-444.23,-973.86,19349.89,5912.69,20901.93,12846.21,...,-119732.25,-123971.21,-129841.27,-116397.08,-129006.87,-116478.45,-103608.16,-104947.13,-80904.51,-87609.43


In [120]:
# Concatenate it all together
report_df = pd.concat([pivot_df, summarized_totals_df, balances_df])


In [121]:
report_df

,2024-01-07,2024-01-14,2024-01-21,2024-01-28,2024-02-04,2024-02-11,2024-02-18,2024-02-25,2024-03-03,2024-03-10,...,2025-10-26,2025-11-02,2025-11-09,2025-11-16,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2025-12-21,2025-12-28
definition_id,,,,,,,,,,,,,,,,,,,,,
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
12,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
13,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
14,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [122]:
# ********************** (4) Prepare report for visual appearance *******************************

# Merge report definition header with report
report_df = pd.merge(definition_df, report_df, left_on="definition_id", right_on="definition_id", how="left")

# Sort based on key value
report_df.sort_values("key", inplace=True)


In [123]:
report_df

,definition_id,key,definition_type,name,2024-01-07 00:00:00,2024-01-14 00:00:00,2024-01-21 00:00:00,2024-01-28 00:00:00,2024-02-04 00:00:00,2024-02-11 00:00:00,...,2025-10-26 00:00:00,2025-11-02 00:00:00,2025-11-09 00:00:00,2025-11-16 00:00:00,2025-11-23 00:00:00,2025-11-30 00:00:00,2025-12-07 00:00:00,2025-12-14 00:00:00,2025-12-21 00:00:00,2025-12-28 00:00:00
0,1,100,NaN,Pamatdarbības naudas plūsma,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,3,102,1.0,Maksājumi piegādātājiem,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
29,31,103,1.0,Maksājumi darbiniekiem,0.00,7421.22,5973.74,6365.74,0.00,0.00,...,18944.96,0.00,0.00,5100.11,7307.81,0.00,26.84,2381.14,0.00,10577.11
3,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,6,199,2.0,Bruto pamatdarbības naudas plūsma,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,7,201,1.0,Izdevumi procentu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,8,202,1.0,Izdevumi nodokļu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,9,203,1.0,Naudas plūsma no ārkārtas posteņiem,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,10,299,2.0,Pamatdarbības neto naudas plūsma,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
# Subtract report formatting in separate dataframe
format_df = report_df["definition_type"]

In [125]:

format_df

0     NaN
1     1.0
2     1.0
29    1.0
3     1.0
4     2.0
5     1.0
6     1.0
7     1.0
8     2.0
9     NaN
10    1.0
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    2.0
19    NaN
20    1.0
21    1.0
22    1.0
23    1.0
24    1.0
25    2.0
26    1.0
27    2.0
28    3.0
Name: definition_type, dtype: float64

In [126]:
# Prepare report_df for visual appearance

report_df.drop(columns=["definition_id", "key", "definition_type"], inplace=True)
report_df.set_index("name", inplace=True)

# Format column headers to show only the date part
report_df.columns = [col.strftime(date_format()) if not pd.isnull(col) else col for col in report_df.columns]



In [127]:
report_df

,"07. Jan, 2024","14. Jan, 2024","21. Jan, 2024","28. Jan, 2024","04. Feb, 2024","11. Feb, 2024","18. Feb, 2024","25. Feb, 2024","03. Mar, 2024","10. Mar, 2024",...,"26. Oct, 2025","02. Nov, 2025","09. Nov, 2025","16. Nov, 2025","23. Nov, 2025","30. Nov, 2025","07. Dec, 2025","14. Dec, 2025","21. Dec, 2025","28. Dec, 2025"
name,,,,,,,,,,,,,,,,,,,,,
Pamatdarbības naudas plūsma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ieņēmumi no preču un pakalpojumu pārdošanas,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Maksājumi piegādātājiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Maksājumi darbiniekiem,0.00,7421.22,5973.74,6365.74,0.00,0.00,0.00,7491.72,67.53,12743.47,...,18944.96,0.00,0.00,5100.11,7307.81,0.00,26.84,2381.14,0.00,10577.11
Pārējie pamatdarbības ieņēmumi un izdevumi,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Bruto pamatdarbības naudas plūsma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Izdevumi procentu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Izdevumi nodokļu maksājumiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Naudas plūsma no ārkārtas posteņiem,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [128]:
report_df.columns


Index(['07. Jan, 2024', '14. Jan, 2024', '21. Jan, 2024', '28. Jan, 2024',
       '04. Feb, 2024', '11. Feb, 2024', '18. Feb, 2024', '25. Feb, 2024',
       '03. Mar, 2024', '10. Mar, 2024',
       ...
       '26. Oct, 2025', '02. Nov, 2025', '09. Nov, 2025', '16. Nov, 2025',
       '23. Nov, 2025', '30. Nov, 2025', '07. Dec, 2025', '14. Dec, 2025',
       '21. Dec, 2025', '28. Dec, 2025'],
      dtype='object', length=104)